In [1]:
import subprocess
import threading
import re

class PowerShellAgent:
    def __init__(self, output_file):
        self.output_file = output_file
        self.proc = None
        self.thread = None
        self.break_flag = threading.Event()
        self.state = 'no running'
    def start_powershell(self):
        if  self.state !=  'no running':
            
            print('Only in state \'no running\' can start listening!')
            return
#         self.proc = subprocess.Popen(["powershell"], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        # 将错误输出重定向到了标准输出，便于一并读取保存
        self.proc = subprocess.Popen(["powershell"], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        self.state = 'running'
    def start_listening(self):

        if self.state != 'running':
            print('Only in state \'running\' can start listening!')
            return
        self.state = 'running and listening'
        self.break_flag.clear()
        self.thread = threading.Thread(target=self.listen_powershell_output)
        self.thread.start()
    def run_powershell_command(self, cmd):
        # 向 PowerShell 进程发送命令
        self.proc.stdin.write(cmd + "\n")
        self.proc.stdin.flush()
    def stop_listening(self):
        if self.state != 'running and listening':
            print('Only in state \'running and listening\' can stop listening!')
            return
        self.state = 'running'
        self.run_powershell_command('echo stop_agent_output')
        self.break_flag.set()
        self.thread.join()
    def listen_powershell_output(self):
        with open(self.output_file, "a") as f:
            while not self.break_flag.is_set():
                line = self.proc.stdout.readline().strip()
                if line.endswith('stop_agent_output'):
                    continue
                f.write(line + "\n")
                f.flush()
    def powershell_shutdown(self):
        if self.state != 'running':
            print('Only in state \'running\' can stop powershell!')
            return
        self.state = 'no running' 
        self.proc.terminate()
    def check_state(self):
        print(f'now agent state:  {self.state}')
    def get_final_output(self,target_log_file=None):
        if not target_log_file:
            target_log_file=self.output_file
        out_put_text = ""
        with open(target_log_file, "r") as f:
            file_content = f.read()
            # 如果指定的cmd为None，将文件中最后一条指令及其输出赋值给out_put_text
            pattern = r"(?s)(?P<command_output>PS\s[^>]+\>\s.+?\n)"  # 匹配最后一条指令及其输出的正则表达式
            match = re.findall(pattern, file_content)
            out_put_text=file_content.split(match[-1])[-1]
        return out_put_text


In [143]:
get_final_output("output.txt")

''

In [3]:
listener = PowerShellAgent("output.txt")

In [4]:
listener.get_final_output("output.txt")

''

In [2]:
listener = PowerShellAgent("output.txt")
listener.start_powershell()
listener.start_listening()



In [11]:
print(listener.get_final_output())



目录: F:\GPT_local\gpt-api-use


Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
d-----         2023/5/21     11:39                .ipynb_checkpoints
d-----         2023/5/23     11:35                test
-a----         2023/5/25     19:33          18182 API测试.ipynb
-a----         2023/5/30     16:41           8310 powershell测试.ipynb
-a----         2023/5/25     19:37          16367 windows防护测试.ipynb
-a----         2023/5/30     16:42           1923 output.txt





In [12]:
listener.check_state()

now agent state:  running and listening


In [4]:
listener.run_powershell_command('ipconfig')

In [13]:
listener.stop_listening()

In [6]:
listener.start_listening()

In [31]:
listener.run_powershell_command('echo $env:userprofile')

In [8]:
listener.run_powershell_command('swdadawd')

In [37]:
listener.run_powershell_command(r'''cd F:\GPT_local\gpt-api-use''')

In [10]:
listener.run_powershell_command(r'''dir''')

In [25]:
listener.run_powershell_command(r'''exit()''')

In [14]:
# 在需要停止监听的时候调用
listener.powershell_shutdown()